## Compare to ground truth (3d DLC annotations)

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import pandas as pd
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *
from DLC_for_WBFM.utils.feature_detection.utils_affine import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Get the 3d bigtiff folder
# Note: actually only 5 frames
bigtiff_folder = r'D:\Old_annotated_DLC\3d_annotations'

btf_fname_red = r'combined_img.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)

annotations_fname = 'CollectedData_Charlie.h5'
annotations_fname = os.path.join(bigtiff_folder, annotations_fname)

## Get two sets of matches: affine, and feature-based


In [4]:
opt = {'start_frame':0,
       'num_frames':5,
      'use_affine_matching':True}
a_matches, a_conf, a_frames = track_neurons_full_video(btf_fname_red, **opt)

opt = {'start_frame':0,
       'num_frames':5,
      'use_affine_matching':False}
f_matches, f_conf, f_frames = track_neurons_full_video(btf_fname_red, **opt)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


 50%|██████████████████████████████████████████                                          | 2/4 [00:20<00:20, 10.18s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:29<00:09,  9.72s/it]

...

 50%|██████████████████████████████████████████                                          | 2/4 [00:19<00:18,  9.17s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:26<00:08,  8.41s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:33<00:00,  8.46s/it]


## Get the DLC annotations


In [80]:
k = (1,2) # Which frames to compare

In [81]:
df = pd.read_hdf(annotations_fname, key="df_with_missing")
df = df['Charlie']

num_neurons = int(len(df.columns)/3)
xyz_gt0 = np.zeros((num_neurons, 3))
xyz_gt1 = np.zeros((num_neurons, 3))
dat_gt = np.zeros((num_neurons, 3))
for i in range(num_neurons):
    name = f"neuron{i}"
    v0 = np.array(df[name].iloc[k[0],:])
    v1 = np.array(df[name].iloc[k[1],:])
    ind = [2,0,1]
    v0, v1 = v0[ind], v1[ind]
    xyz_gt0[i,:] = v0
    xyz_gt1[i,:] = v1
    dat_gt[i,:] = v1-v0

In [82]:
# xyz_gt0

## Make these into point clouds, with lines for matches

In [83]:
# First, the automated matches

a0_unmatched = a_frames[k[0]].neuron_locs
a1_unmatched = a_frames[k[1]].neuron_locs
this_match, this_conf = a_matches[k], a_conf[k]
# all_m, all_c, to_remove_ind = keep_best_match(this_match, this_conf)

a_xyz = np.zeros((len(this_match), 3), dtype=np.float32) # Start point
a_dat = np.zeros((len(this_match), 3), dtype=np.float32) # Difference vector
for m, match in enumerate(this_match):
    v0 = a0_unmatched[match[0]]
    v1 = a1_unmatched[match[1]]
    a_xyz[m, :] = v0
    a_dat[m, :] = v1 - v0

f0_unmatched = f_frames[k[0]].neuron_locs
f1_unmatched = f_frames[k[1]].neuron_locs
this_match, this_conf = f_matches[k], f_conf[k]
# all_m, all_c, to_remove_ind = keep_best_match(this_match, this_conf)

f_xyz = np.zeros((len(this_match), 3), dtype=np.float32) # Start point
f_dat = np.zeros((len(this_match), 3), dtype=np.float32) # Difference vector
for m, match in enumerate(this_match):
    v0 = f0_unmatched[match[0]]
    v1 = f1_unmatched[match[1]]
    f_xyz[m, :] = v0
    f_dat[m, :] = v1 - v0

In [84]:
# Affine
pc0 = build_neuron_tree(a_xyz, False)[1]
pc1 = build_neuron_tree(a_xyz+a_dat, False)[1]

affine_lines = build_line_set_from_matches(pc0, pc1, matches=None, color=[0,0,1])
affine_lines.translate(np.array([0.0,0.0,0.5]))

# Feature (same points, ordered differently)
pc0 = build_neuron_tree(f_xyz, False)[1]
pc1 = build_neuron_tree(f_xyz+f_dat, False)[1]

feature_lines = build_line_set_from_matches(pc0, pc1, matches=None, color=[1,0,0])
# Offset for visualization
feature_lines.translate(np.array([0.0,0.5,0.0]))

# Define full point clouds
pc0 = build_neuron_tree(a0_unmatched, False)[1]
pc0.paint_uniform_color([0,0,0])
pc1 = build_neuron_tree(a1_unmatched, False)[1]
pc1.paint_uniform_color([0.5,0.5,0.5])

PointCloud with 84 points.

In [85]:

pc0_gt = build_neuron_tree(xyz_gt0, False)[1]
pc1_gt = build_neuron_tree(xyz_gt1, False)[1]

gt_lines = build_line_set_from_matches(pc0_gt, pc1_gt, matches=None, color=[0,1,0])

## Finally, visualize

In [86]:
o3d.visualization.draw_geometries([pc0, pc1, affine_lines, feature_lines, gt_lines])